Scrapping the data webpage

In [1]:
# install dependencies
!pip install pandas
!pip install selenium
!pip install bs4

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because nor

In [2]:
# imports
from bs4 import BeautifulSoup

# selenium will be used to scrap dynamic content of the webpage source of our data
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# get the webpage contents in html

url= "https://cycling.data.tfl.gov.uk"

cap = DesiredCapabilities().FIREFOX
cap["marionette"] = False

options = FirefoxOptions()
options.add_argument("--headless")

browser = webdriver.Firefox(capabilities=cap, executable_path=GeckoDriverManager().install(), options=options)
browser.get(url)

wait = WebDriverWait(browser, 20)
html = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/table/tbody/tr[1]/td[1]')))

html_element= browser.page_source




====== WebDriver manager ======
Current firefox version is 96.0
Get LATEST geckodriver version for 96.0 firefox
Driver [/home/hrc/.wdm/drivers/geckodriver/linux64/v0.30.0/geckodriver] found in cache
/tmp/ipykernel_121949/2524136705.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(capabilities=cap, executable_path=GeckoDriverManager().install(), options=options)
/tmp/ipykernel_121949/2524136705.py:11: DeprecationWarning: capabilities and desired_capabilities have been deprecated, please pass in a Service object
  browser = webdriver.Firefox(capabilities=cap, executable_path=GeckoDriverManager().install(), options=options)


In [4]:
# scrap the html contents
bsoup= BeautifulSoup(html_element, "html.parser")

In [5]:
# find the relevant files with their links
table= bsoup.find('table')
tbody= table.find('tbody')
folder_name= "usage-stats/"
capture_files= False
year= 2021
filetype= 'csv'
extracted_files= {}

for row in tbody.find_all('tr'):
    columns= row.find_all('td')

    if capture_files == False:
        col_values= [col.text.strip() for col in columns]

        if col_values[0] == folder_name:
            capture_files= True
            continue

    else:
        col= columns[0]
        filename= col.text.strip()
        
        if not filename.endswith(f'{year}.{filetype}'):
            continue
        
        # extract the date
        filename_without_extension= filename.replace(f'.{filetype}', '') 
        filename_last_date= filename_without_extension.split('-')[-1]
        extracted_files[filename_last_date]= col.a['href']

print('Display 2 items in the dictionary')
print(dict(list(extracted_files.items())[0:2]))

Display 2 items in the dictionary
{'05Jan2021': 'https://cycling.data.tfl.gov.uk/usage-stats/246JourneyDataExtract30Dec2020-05Jan2021.csv', '12Jan2021': 'https://cycling.data.tfl.gov.uk/usage-stats/247JourneyDataExtract06Jan2021-12Jan2021.csv'}
